In [1]:
%load_ext autoreload
%autoreload 2

import pymc3 as pm
import numpy as np
import pandas as pd
import theano
import theano.tensor as tt
import pickle
import functools
import itertools

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.style.use('seaborn')

from model import GPTrendIndividualModel, GPTrendHierModel
from util import dataset_from_df
from analyse import metric1, metric2, metric_mse, metric_var

pd.set_option('display.max_rows', 500)

%matplotlib inline

In [2]:
nut = ['STARCH', 'SUGAR', 'FIBC', 'FAT', 'PROT']
user_profile = ['sex', 'age', 'height', 'weight', 'bmi']
low, high = 0, 24*60

# IC

In [6]:
models = [
#     'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-ind-p13-ls-testday-days2-nostd-featlog-notime',
    'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog-notime',
#     'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog',
#     'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog-cov',
    'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog-cov0.05',
]
cmp_dict = {}
for mn in models:
    name = ','.join(['hier' if 'hier' in mn else 'ind',
                 '' if 'notime' in mn else 'time',
                 'd1' if 'days1' in mn else '',
                 'cov' if 'cov' in mn else '',
                ])
    store = pickle.load(open('trace/{}.pkl'.format(mn), 'rb'))
    m = store['model']
    trace = store['trace']
    m.model.name = name
    cmp_dict[m.model] = trace

In [7]:
# cov0.05
df_comp_WAIC = pm.compare(cmp_dict, ic='LOO')
df_comp_WAIC

/u/67/zhangg1/unix/.local/lib/python3.6/site-packages/pymc3/stats.py:162: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)
/u/67/zhangg1/unix/.local/lib/python3.6/site-packages/pymc3/stats.py:292: UserWarning: Estimated shape parameter of Pareto distribution is
        greater than 0.7 for one or more samples.
        You should consider using a more robust model, this is because
        importance sampling is less likely to work well if the marginal
        posterior and LOO posterior are very different. This is more likely to
        happen with a non-robust model and highly influential observations.
  happen with a non-robust model and highly influential observations.""")


,LOO,pLOO,dLOO,weight,SE,dSE,shape_warn
"hier,time,,cov",2994.98,465.47,0,0.91,333.7,0,1
"hier,,,",3587.87,214.62,592.89,0.09,317.28,227.51,1


In [5]:
df_comp_WAIC = pm.compare(cmp_dict, ic='LOO')
df_comp_WAIC

/home/zhangg1/.local/lib/python3.6/site-packages/pymc3/stats.py:292: UserWarning: Estimated shape parameter of Pareto distribution is
        greater than 0.7 for one or more samples.
        You should consider using a more robust model, this is because
        importance sampling is less likely to work well if the marginal
        posterior and LOO posterior are very different. This is more likely to
        happen with a non-robust model and highly influential observations.
  happen with a non-robust model and highly influential observations.""")


,LOO,pLOO,dLOO,weight,SE,dSE,shape_warn
"hier,time,,",2869.91,342.24,0,0.25,265.09,0,1
"hier,time,,cov",2948.41,419.88,78.5,0.67,350.12,114.86,1
"ind,,,",3549.64,246.55,679.74,0,318.8,193.02,1
"hier,,,",3587.87,214.62,717.96,0.08,317.28,194.7,1


# U-test

In [ ]:
from scipy.stats import mannwhitneyu


def get_mse_samples(model_file, delay=False, nppc=100):
    print(model_file)
    
    delay = delay and not 'notime' in model_file
    
    store = pickle.load(open('trace/{}.pkl'.format(model_file), 'rb'))
    store_ppc = pickle.load(open('trace/{}_ppc.pkl'.format(model_file), 'rb'))

    ids = store['ids']
    m = store['model']
    trace = store['trace']
    ppc = store_ppc['ppc']
    
    if delay:
        ppc_delay = m.predict(trace, nppc, suffix='delay', test_only=True, delay=True)

    samples = []
    for i, id_ in enumerate(ids):
        trend_t = ppc['trend_test{}{}'.format('', i)]
        if delay:
            tr_t = ppc_delay['treated_test{}{}'.format(i,'delay')]
        else:
            tr_t = ppc['treated_test{}'.format(i)]
        
        samples.append([metric_mse(m.yt[i], trend_t[j], tr_t[j], m.tt[i], m.txt[i], window=(60,180), active=False, realtime=False) 
                        for j in range(nppc)])
        
    return samples

In [ ]:
# Lognormal cov0.05
models = [
    'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog-cov0.05',
    'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog',
]
model_tar = 'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog-notime'
delay = False

In [ ]:
samples_tar = get_mse_samples(model_tar, delay)
sam_tar = np.mean(np.array(samples_tar), axis=0)


PVs = {}
for mf in models:
    samples = get_mse_samples(mf, delay=delay)
    sam = np.mean(np.array(samples), axis=0)
    pv = mannwhitneyu(sam_tar, sam, use_continuity=False, alternative='greater')[1]

    print(mf, pv)

# Metric

In [76]:
models = [
    'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-ind-p13-ls-testday-days2-nostd-featlog-notime',
    'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog-notime',
    'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog-cov',    
    'v12302145-nuts-sigHN01_softl0h0_ls10HN10_hv0_bhHN3_blHN3-I_wlN5_whN5-H_wlN5HN3_whN5HN1-joint-uspgp-pol0-hier-p13-ls-testday-days2-nostd-featlog',
]
delay = False
mse_select = True

In [10]:
mtbl = pd.DataFrame()
for mf_ in models:
    M1, M2, M3, M4, M5 = [], [], [], [], []
    name = ','.join(['hier' if 'hier' in mf_ else 'ind',
                     '' if 'notime' in mf_ else 'time',
                     #'^2' if 'poly2' in mf_ else '',
                     'd1' if 'days1' in mf_ else '',
                     'cov' if 'cov' in mf_ else '',
                    ])
    
    #for mf in [mf_, mf_.replace('testp01', 'testday')]:
    for mf in [mf_]:
        store = pickle.load(open('trace/{}.pkl'.format(mf), 'rb'))
        store_ppc = pickle.load(open('trace/{}_ppc.pkl'.format(mf), 'rb'))

        ids = store['ids']
        dfs = store['dfs']
        data = [dataset_from_df(df, nut) for df in dfs]
        prof = np.vstack([get_user_profile(df, user_profile) for df in dfs])

        t_s = np.linspace(low, high*3, 500)

        m = store['model']
        trace = store['trace']
        ppc = store_ppc['ppc']
        #trends = store_ppc['trends']
        if delay and 'time' in name:
            ppc_delay = m.predict(trace, 100, suffix='delay', test_only=True, delay=True)

        for i, id_ in enumerate(ids):
            trend = ppc['trend{}{}'.format('', i)].mean(axis=0)
            trend_t = ppc['trend_test{}{}'.format('', i)].mean(axis=0)
            tr = ppc['treated{}{}'.format('', i)].mean(axis=0)
            if delay and 'time' in name:
                tr_t = ppc_delay['treated_test{}{}'.format(i,'delay')].mean(axis=0)
            else:
                tr_t = ppc['treated_test{}'.format(i)].mean(axis=0)

            M1.append(metric1(m.y[i], trend, tr))
            M2.append(metric2(m.y[i], trend, tr))
            M3.append(metric_mse(m.y[i], trend, tr, m.t[i], m.tx[i], window=None, active=False, realtime=False))
            M4.append(metric_mse(m.yt[i], trend_t, tr_t, m.tt[i], m.txt[i], window=(60,180), active=False, realtime=False))
            M5.append(metric_var(m.yt[i], trend_t, tr_t, m.tt[i], m.txt[i], window=(60,180)))
    
    if mse_select:
        idx_mse = np.array(M2) > 0.1
    else:
        idx_mse = np.array(M2) > 0.

    M1.append(np.mean(M1))
    M2.append(np.mean(M2))
    M3.append(np.mean(M3))
    M4.append(np.mean(np.array(M4)[idx_mse]))
    M5.append(np.mean(np.array(M5)[idx_mse]))
    mtbl[name] = M1 + M2 + M3 + M4 + M5
    
mtbl['ID'] = (ids.tolist()+['Ave']) * 5 # Caution! Only works in notebooks
n = len(ids)+1
# mtbl.index = ['V(trd-y)/V(trd)']*n + ['V(tr)/V(trd-y)']*n + ['MSE']*n + ['Future MSE']*n
mtbl.index = ['V(trd)/V(y)']*n + ['V(trd+tr)/V(y)-M1']*n + ['MSE']*n + ['Future MSE']*n + ['Future Var%']*n

/work/modules/Ubuntu/14.04/amd64/common/anaconda3/latest/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)
100%|██████████| 100/100 [03:51<00:00,  1.82s/it]


In [ ]:
mtbl

In [ ]:
# with open('metric.tex','w') as tf:
#     tf.write(mtbl.to_latex())